In [1]:
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import Tool

# ----------------- TOOLS -----------------
def search_database(query: str) -> str:
    if "abcd" in query.lower():
        return "‘ABCD’ पुस्तकको मूल्य रु ५०० हो।"
    return "NOT_FOUND"

def create_ticket(name: str, date: str) -> str:
    return f"टिकट {name} को लागि {date} मा बुक गरियो।"

def create_booking(item: str, quantity: int) -> str:
    return f"{item} को लागि {quantity} बुकिङ सफल भयो।"

search_tool = Tool.from_function(
    func=search_database,
    name="search_database",
    description="Search book price or details in the database",
    args_schema={"type": "object", "properties": {"query": {"type": "string"}}, "required": ["query"]}
)

ticket_tool = Tool.from_function(
    func=create_ticket,
    name="ticket",
    description="Create a ticket",
    args_schema={"type": "object", "properties": {"name": {"type": "string"}, "date": {"type": "string"}}, "required": ["name","date"]}
)

booking_tool = Tool.from_function(
    func=create_booking,
    name="booking",
    description="Create a booking",
    args_schema={"type": "object", "properties": {"item": {"type": "string"}, "quantity": {"type": "integer"}}, "required": ["item","quantity"]}
)

# ----------------- AGENTS -----------------
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

search_agent = create_react_agent(llm, [search_tool])
action_agent = create_react_agent(llm, [ticket_tool, booking_tool])
reply_agent = ChatOpenAI(model="gpt-4o-mini", temperature=0)
decision_agent = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ----------------- STATE -----------------
class ConversationState(TypedDict):
    user_input: str
    decision: Literal["search", "action", "done", "error"]
    intermediate: str
    reply: str
    tool_call_log: list
    error: Optional[str]

# ----------------- NODES -----------------
def decision_node(state: ConversationState) -> ConversationState:
    prompt = f"""यो प्रश्नको लागि सही मार्ग निर्धारण गर्नुहोस्: '{state['user_input']}'
    
    निर्णय नियमहरू:
    - 'search': पुस्तकको मूल्य वा विवरण खोज्ने प्रश्नहरू
    - 'action': टिकट बुकिङ, आरक्षण, वा कार्यहरू गर्ने प्रश्नहरू
    
    केवल 'search' वा 'action' मात्र जवाफ दिनुहोस्।"""
    
    decision = decision_agent.invoke([{"role": "user", "content": prompt}]).content.strip().lower()
    state["decision"] = decision if decision in ["search", "action"] else "error"
    return state

def search_node(state: ConversationState) -> ConversationState:
    try:
        resp = search_agent.invoke({"messages": [("user", state["user_input"])]})
        content = resp["messages"][-1].content
        if "NOT_FOUND" in content:
            state["decision"] = "action"
        else:
            state["intermediate"] = content
            state["decision"] = "done"
            state.setdefault("tool_call_log", []).append({
                "tool": "search_database", 
                "output": content
            })
    except Exception as e:
        state["decision"] = "error"
        state["error"] = f"Search failed: {str(e)}"
    return state

def action_node(state: ConversationState) -> ConversationState:
    try:
        resp = action_agent.invoke({"messages": [("user", state["user_input"])]})
        content = resp["messages"][-1].content
        state["intermediate"] = content
        state["decision"] = "done"
        for msg in resp["messages"]:
            if hasattr(msg, "tool") and msg.tool:
                state.setdefault("tool_call_log", []).append({
                    "tool": msg.tool,
                    "tool_input": msg.tool_input,
                    "output": msg.content
                })
    except Exception as e:
        state["decision"] = "error"
        state["error"] = f"Action failed: {str(e)}"
    return state

def reply_node(state: ConversationState) -> ConversationState:
    reply = reply_agent.invoke([{"role": "user", "content": f"यसलाई छोटकरीमा नेपालीमा जवाफ देऊ: {state['intermediate']}"}]).content
    state["reply"] = reply
    return state

def error_node(state: ConversationState) -> ConversationState:
    state["reply"] = f"माफ गर्नुहोस्, समस्या आयो: {state.get('error', 'अज्ञात त्रुटि')}"
    return state

# ----------------- GRAPH -----------------
graph = StateGraph(ConversationState)

# Add all nodes
graph.add_node("decision", decision_node)
graph.add_node("search", search_node)
graph.add_node("action", action_node)
graph.add_node("reply", reply_node)
graph.add_node("error", error_node)

# Set starting point
graph.add_edge(START, "decision")

# Decision node conditional routing
def route_after_decision(state: ConversationState) -> Literal["search", "action", "error"]:
    if state["decision"] == "search":
        return "search"
    elif state["decision"] == "action":
        return "action"
    else:
        return "error"

graph.add_conditional_edges("decision", route_after_decision)

# Search node conditional routing
def route_after_search(state: ConversationState) -> Literal["action", "reply", "error"]:
    if state["decision"] == "action":
        return "action"
    elif state["decision"] == "done":
        return "reply"
    else:
        return "error"

graph.add_conditional_edges("search", route_after_search)

# Action node conditional routing
def route_after_action(state: ConversationState) -> Literal["reply", "error"]:
    if state["decision"] == "done":
        return "reply"
    else:
        return "error"

graph.add_conditional_edges("action", route_after_action)

# Final edges
graph.add_edge("reply", END)
graph.add_edge("error", END)

workflow = graph.compile()

# ----------------- EXAMPLE USAGE -----------------
if __name__ == "__main__":
    # Test the workflow
    test_inputs = [
        "abcd पुस्तकको मूल्य कति हो?",
        "टिकट बुक गर्नुहोस्",
        "किताबको बुकिङ गर्नुहोस्"
    ]
    
    for user_input in test_inputs:
        print(f"\nInput: {user_input}")
        result = workflow.invoke({
            "user_input": user_input,
            "decision": "",
            "intermediate": "",
            "reply": "",
            "tool_call_log": [],
            "error": None
        })
        print(f"Output: {result['reply']}")


Input: abcd पुस्तकको मूल्य कति हो?
Output: 'ABCD' पुस्तकको मूल्य रु ५०० हो।

Input: टिकट बुक गर्नुहोस्
Output: कृपया तपाईंको नाम र मिति दिनुहोस्।

Input: किताबको बुकिङ गर्नुहोस्
Output: कृपया तपाईं कुन प्रकारको किताब बुक गर्न चाहनुहुन्छ र कति प्रतिहरू बुक गर्न चाहनुहुन्छ भन्ने जानकारी दिनुहोस्।
